# Test results for original dataset
Test the performance of the obtained trained models on the **original dataset**. Create a table with erformance scores.

## Import useful packages

In [1]:
# Generic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import pickle

In [2]:
# Scikit-learn for vectorizers and performance metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_curve, auc, f1_score, make_scorer, precision_recall_curve, matthews_corrcoef

In [3]:
# Keras preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import load_model

In [4]:
# NLTK for natural language processing
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
# Custom helper-functions script (supplied to Colab manually)
import utils as uu

## Load data and assess performance

In [6]:
# Define file names
train_set_file = "train_set_imb_0.csv"
test_set_file  = "test_set_imb_0.csv"

In [7]:
# Set model-specific load and presentation parameters
models = []

# LR model
models.append({
    "name":             "LR",
    "linecolor":        'green',
    "linestyle":        '-', 
    "marker":           '^',
    "model_file":       "model_lr_imb_0.pckl",
    "vectorizer":       "TFIDF",
    "is_dl_model":      False,
    "is_lex_model":     False,
    "max_features":     212435
        })

# RF model
models.append({
    "name":             "RF",
    "linecolor":        'orange',
    "linestyle":        '-', 
    "marker":           'v',
    "model_file":       "model_rf_imb_0.pckl",
    "vectorizer":       "COUNT",
    "is_dl_model":      False,
    "is_lex_model":     False,
    "max_features":     89403
        })

# SVM model
models.append({
    "name":             "SVM",
    "linecolor":        'magenta',
    "linestyle":        '-', 
    "marker":           '+',
    "model_file":       "model_svm_imb_0.pckl",
    "vectorizer":       "TFIDF",
    "is_dl_model":      False,
    "is_lex_model":     False,
    "max_features":     2710
        })

# LSTM model
models.append({
    "name":             "LSTM",
    "linecolor":        'purple',
    "linestyle":        '--',
    "marker":           'None',
    "model_file":       "model_lstm_imb_0.h5",
    "vectorizer":       "KERAS",
    "is_dl_model":      True,
    "is_lex_model":     False,
    "max_features":     100000
        })

# FCNN model
models.append({
    "name":             "FCNN",
    "linecolor":        'deepskyblue',
    "linestyle":        '--', 
    "marker":           'o',
    "model_file":       "model_fcnn_imb_0.h5",
    "vectorizer":       "KERAS",
    "is_dl_model":      True,
    "is_lex_model":     False,
    "max_features":     100000
        })

# NB model
models.append({
    "name":             "NB",
    "linecolor":        'lime',
    "linestyle":        '-', 
    "marker":           'x',
    "model_file":       "model_nb_imb_0.pckl",
    "vectorizer":       "COUNT",
    "is_dl_model":      False,
    "is_lex_model":     False,
    "max_features":     235808
        })

# CNN model
models.append({
    "name":             "CNN",
    "linecolor":        'red',
    "linestyle":        '--', 
    "marker":           '.',
    "model_file":       "model_cnn_imb_0.h5",
    "vectorizer":       "KERAS",
    "is_dl_model":      True,
    "is_lex_model":     False,
    "max_features":     100000
        })

# Lexicon model
models.append({
    "name":             "Lexicon",
    "linecolor":        'black',
    "linestyle":        '-.', 
    "marker":           'd',
    "model_file":       "manual",
    "vectorizer":       "NONE",
    "is_dl_model":      False,
    "is_lex_model":     True,
    "max_features":     100000
        })

# KNN model
models.append({
    "name":             "KNN",
    "linecolor":        'blue',
    "linestyle":        '-', 
    "marker":           'None',
    "model_file":       "model_knn_imb_0.pckl",
    "vectorizer":       "TFIDF",
    "is_dl_model":      False,
    "is_lex_model":     False,
    "max_features":     100000
        })

In [8]:
# Set parameters for tokenization
max_words = 5000
max_len = 55

In [9]:
# Define a UL lexicon for label "1"
dict1 = ['climate',
 'climatechange',
 'globalwarming',
 'agw',
 'climaterealists']

In [10]:
# Compute performance

# Load datasets
train_set = pd.read_csv(train_set_file)
test_set = pd.read_csv(test_set_file)

# Preprocess texts
train_set['text'] = train_set['text'].apply(str)
train_set['text'] = train_set['text'].apply(uu.preprocess_text)
test_set['text'] = test_set['text'].apply(str)
test_set['text'] = test_set['text'].apply(uu.preprocess_text)

# Get features and labels
texts_train = copy.deepcopy(train_set['text'])
labels_train = copy.deepcopy(train_set['is_about_cc'])
texts_test = copy.deepcopy(test_set['text'])
labels_test = copy.deepcopy(test_set['is_about_cc'])

# Performance dataframe
perf = []

for model_idx in range(len(models)):

  print("model_idx = " + str(model_idx))
  
  # Load model
  if (models[model_idx]["is_dl_model"] == True):
    model = load_model(models[model_idx]["model_file"])
  elif (models[model_idx]["is_lex_model"] == False):
    model = pickle.load(open(models[model_idx]["model_file"], 'rb'))

  if (models[model_idx]["vectorizer"] == "KERAS"):
    # Tokenize features
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(texts_train)
    sequences_train = tokenizer.texts_to_sequences(texts_train)
    features_train = sequence.pad_sequences(sequences_train, maxlen=max_len, padding='post', truncating='post')
    sequences_test = tokenizer.texts_to_sequences(texts_test)
    features_test = sequence.pad_sequences(sequences_test, maxlen=max_len, padding='post', truncating='post')
  elif (models[model_idx]["vectorizer"] == "TFIDF"):
    # Vectorize features
    vectorizer = TfidfVectorizer(max_features=models[model_idx]["max_features"], lowercase=True, analyzer='word', dtype=np.float32)
    vectorizer.fit(texts_train)
    features_train = vectorizer.transform(texts_train)
    features_test = vectorizer.transform(texts_test)
  elif (models[model_idx]["vectorizer"] == "COUNT"):
    vectorizer = CountVectorizer()
    vectorizer.fit(texts_train)
    features_train = vectorizer.transform(texts_train)
    features_test = vectorizer.transform(texts_test)
    
  # Compute performance metrics

  # Predicted labels
  if (models[model_idx]["is_lex_model"] == True):
    pred_labels = [1 if any(word in text.split() for word in dict1) else 0 for text in texts_test]
  else:
    pred_labels = (model.predict(features_test) > 0.5).astype(int)

  # Accuracy, precision, recall and F1 score
  acc, prec, rec, f1 = uu.compute_perf_metrics(labels_test, pred_labels)

  # Prediction scores
  if (models[model_idx]["name"] == "LR") or (models[model_idx]["name"] == "SVM"):
    pred_scores = model.decision_function(features_test)
  elif (models[model_idx]["name"] == "RF") or (models[model_idx]["name"] == "NB") or (models[model_idx]["name"] == "KNN"):
    pred_scores = model.predict_proba(features_test)[:, 1]
  elif (models[model_idx]["name"] == "Lexicon"):
    pred_scores = [uu.occurrence_counter(text.split(), dict1) for text in texts_test]
  elif (models[model_idx]["name"] == "CNN") or (models[model_idx]["name"] == "FCNN") or (models[model_idx]["name"] == "LSTM"):
    pred_scores = model.predict(features_test).ravel()

  # Area under ROC curve
  fpr, tpr = uu.compute_roc(labels_test, pred_scores)
  roc_auc = auc(fpr, tpr)

  # Area uner PR curve
  precs, recs = uu.compute_pr(labels_test, pred_scores)
  pr_auc = auc(recs, precs)

  # Matthews correlation coeficient
  mcc = matthews_corrcoef(labels_test, pred_labels)

  # Gather all metrics
  perf.append([models[model_idx]["name"], acc, prec, rec, f1, roc_auc, pr_auc, mcc])

# Combine into a dataframe
df_perf = pd.DataFrame(perf, columns=['Method', 'Accuracy', 'Precision', 'Recall', 'F1 score', 'AUC ROC', 'AUC PR', 'MCC'])

model_idx = 0
model_idx = 1
model_idx = 2
model_idx = 3
model_idx = 4
model_idx = 5
model_idx = 6
model_idx = 7
model_idx = 8


In [11]:
# Performance comparison
df_perf

,Method,Accuracy,Precision,Recall,F1 score,AUC ROC,AUC PR,MCC
0,LR,0.957077,0.829268,0.531250,0.647619,0.913945,0.694443,0.643570
1,RF,0.952436,0.716981,0.593750,0.649573,0.924048,0.685173,0.627497
2,SVM,0.941995,0.606061,0.625000,0.615385,0.881050,0.641829,0.584106
3,LSTM,0.944316,0.642857,0.562500,0.600000,0.863565,0.591703,0.571686
4,FCNN,0.938515,0.593220,0.546875,0.569106,0.890488,0.615837,0.536574
5,NB,0.937355,0.586207,0.531250,0.557377,0.919447,0.617646,0.524492
6,CNN,0.935035,0.568966,0.515625,0.540984,0.891115,0.605069,0.506828
7,Lexicon,0.957077,0.909091,0.468750,0.618557,0.732525,0.709352,0.635333
8,KNN,0.938515,1.000000,0.171875,0.293333,0.887287,0.598238,0.401461


In [12]:
# Save into a file
df_perf.to_csv("tab_metrics.csv", index=False)